# String Simulation Analysis

This notebook will help you analyse the convergence of the string-method and if you are lucky extract a nice free energy surface. 

In [ ]:
import numpy as np
import glob as glob
from math import ceil
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from MDAnalysis.analysis.align import AlignTraj
import MDAnalysis as mda
import nglview as nv
import pickle
import logging
import sys
import os

logging.getLogger("matplotlib").setLevel(logging.ERROR)
logging.getLogger("pyemma").setLevel(logging.NOTSET)

In [ ]:
def colorbar(mappable, cmap, norm, label0, size=10):
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap, norm=norm)
    cbar.set_label(label0, size=size)
    return cbar

In [ ]:
plt.rcParams["axes.facecolor"] = "#f9f9fb"
plt.rcParams["grid.color"] = "white"
plt.rcParams["grid.linestyle"] = "-"
plt.rcParams["grid.linewidth"] = 2
plt.rcParams["axes.grid"] = True
plt.rcParams["lines.solid_capstyle"] = "round"

In [ ]:
def natural_sort(l):
    """
    Takes as input a list l of strings and sorts it with natural order.
      Parameters
      ----------
      l: list of strings.
      Returns
      -------
      l sorted
    """
    from re import split

    assert isinstance(l, list), "l is not a list!"
    for i in l:
        assert isinstance(i, str), "List contains non-string elements."
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in split("([0-9]+)", key)]
    return sorted(l, key=alphanum_key)

# String Convergence Analysis

## Extract CVs

In the cell bellow you can select which will be the simulation directory (in case this notebook is elsewhere). If the notebook is in the simulation directory just leave it as ".".

In [ ]:
simulation_directory = "."
os.chdir(simulation_directory)
os.getcwd()

Load the strings in the `strings` variable.

In [ ]:
files = natural_sort(glob.glob("./strings/string*txt"))

In [ ]:
strings = np.array([np.loadtxt(file).T for file in files])

In [ ]:
with open("cv.pkl", "rb") as file:
    cvs, ndx_groups = pickle.load(file)

In [ ]:
print("String details")
print("")
print(f"Number of string: {strings.shape[0]}")
print(f"Number of cvs: {strings.shape[1]}")
print(f"Number of beads per string: {strings.shape[2]}")

# Analyze string convergence
In these next plots you will be able to study the convergence of the string. At convergence the strings should be oscillating around an equilibrium position and not drift over the different iterations.

## Strings as a function of time
In this plot we can see the evolution of each string CV as function of the timeration number separatelly.

You can change two parameters in these plots the `start_iteration` before which all data is not plotted and the `n_average` which is the number of strings iterations to average in one block of strings. This is done in order to cancel some of the noisyness in the representation, to reduce the number of strings in the plot and to see more clearly if there is average drift.

+ borrar último string
+ probar con el GPCR

In [ ]:
start_iteration = 50
n_average = 5


n_plots = strings.shape[1]
n_strings = strings.shape[0]
fig, ax = plt.subplots(ceil(n_plots / 2), 2, figsize=(20, 8 * ceil(n_plots / 2)))
ax = ax.flatten()
cmap = plt.cm.viridis_r
n_colors = (n_strings - start_iteration) // n_average + 1
colors = cmap(np.linspace(0, 1, n_colors))  # yellow to blue
norm = mpl.colors.Normalize(vmin=start_iteration, vmax=n_strings)

for i, a in enumerate(ax[:n_plots]):
    a.plot(strings[0, i, :], ls=":", marker=".", label="string0", color="k")
    for jj, j in enumerate(range(start_iteration, n_strings, n_average)):
        string = np.mean(strings[j : j + n_average, i, :], axis=0)
        a.plot(string, ls="-", marker="o", color=colors[jj])
    av = np.mean(strings[start_iteration:, i, :], axis=0)
    std = np.std(strings[start_iteration:, i, :], axis=0)
    a.fill_between(
        np.arange(len(av)),
        av + std,
        av - std,
        alpha=0.4,
        label=f"std(string{start_iteration}-{n_strings})",
    )
    a.plot(
        av,
        ls="-",
        marker=".",
        color="k",
        label=f"mean(string{start_iteration}-{n_strings})",
    )
    a.set_ylabel(
        f"{list(ndx_groups.keys())[2*i]} - {list(ndx_groups.keys())[2*i+1]} (nm)",
        size=18,
        labelpad=16,
    )
    a.set_xlabel("bead number", size=15, labelpad=13)
    a.set_xlim(left=0, right=strings.shape[2] - 1)
    a.xaxis.set_minor_locator(MultipleLocator(1))
    a.xaxis.set_major_locator(MultipleLocator(1))
    a.yaxis.set_minor_locator(MultipleLocator(0.1))
    a.yaxis.set_major_locator(MultipleLocator(0.1))
    a.grid(which="minor")
    a.tick_params(axis="y", labelsize=14)
    a.tick_params(axis="x", labelsize=11)
    if i % 2 != 0:
        a.legend()
        cbar = colorbar(a, cmap, norm, "iteration number", 20)
if n_plots % 2:
    fig.delaxes(ax[-1])

## Evolution of beads with iterations

In this plot the trajectory of beads cvs over iterations is presented. You can observe if the beads are stable or drifting in some direction. In this plot you can choose one parameter, `n_rolling_average`, which determines the size of the rolling average window applied to smooth-out the noisyness.

In [ ]:
n_rolling_average = 20

n_strings = strings.shape[0]
n_cvs = strings.shape[1]
n_beads = strings.shape[2]
cmap = plt.cm.viridis_r
colors = cmap(np.linspace(0, 1, n_beads - 1))  # yellow to blue
norm = mpl.colors.Normalize(vmin=0, vmax=n_beads - 1)
fig, ax = plt.subplots(ceil(n_cvs / 2), 2, figsize=(20, 8 * ceil(n_cvs / 2)))
ax = ax.flatten()
for i in range(n_cvs):
    for j in range(1, n_beads - 1):
        y = np.convolve(
            strings[:, i, j],
            np.ones((n_rolling_average,)) / n_rolling_average,
            mode="valid",
        )
        x = np.arange(n_strings)[n_rolling_average - 1 :]
        ax[i].plot(x, y, color=colors[j])
        ax[i].set_ylabel(
            f"{list(ndx_groups.keys())[2*i]} - {list(ndx_groups.keys())[2*i+1]} (nm)",
            size=18,
            labelpad=16,
        )
        ax[i].set_xlabel("iteration number", size=15, labelpad=13)
        ax[i].yaxis.set_minor_locator(MultipleLocator(0.1))
        ax[i].yaxis.set_major_locator(MultipleLocator(0.1))
        ax[i].grid(which="minor")
        ax[i].tick_params(axis="y", labelsize=14)
        ax[i].tick_params(axis="x", labelsize=11)
    if i % 2 != 0:
        ax[i].legend()
        cbar = colorbar(ax[i], cmap, norm, "bead number", 20)
if n_plots % 2:
    fig.delaxes(ax[-1])

## Evolution over iterations of pairs of cvs
In this plot you can see the evolution of the string with iterations projected onto two chosen cvs. The plots are just like in the previous section.

You can choose your pair of cvs with the variables `cv_0` and `cv_1`.

In [ ]:
print("CV numbers")
ndx_groups_list = list(ndx_groups)
for i, cv in enumerate(cvs):
    print(f"CV {i} is {ndx_groups_list[cv[0]-1]} - {ndx_groups_list[cv[1]-1]}")

In [ ]:
cv_0 = 3
cv_1 = 0

In [ ]:
start_iteration = 50
n_average = 10

n_strings = strings.shape[0]
fig, ax = plt.subplots(1, 1, figsize=(10, 7))
cmap = plt.cm.viridis_r
n_colors = (n_strings - start_iteration) // n_average + 1
colors = cmap(np.linspace(0, 1, n_colors))  # yellow to blue
norm = mpl.colors.Normalize(vmin=start_iteration, vmax=n_strings)
ax.plot(
    strings[0, cv_0, :],
    strings[0, cv_1, :],
    ls=":",
    marker=".",
    label="string0",
    color="k",
)
av_0 = np.mean(strings[start_iteration:, cv_0, :], axis=0)
std_0 = np.std(strings[start_iteration:, cv_0, :], axis=0)
av_1 = np.mean(strings[start_iteration:, cv_1, :], axis=0)
std_1 = np.std(strings[start_iteration:, cv_1, :], axis=0)
ax.plot(
    av_0,
    av_1,
    ls="-",
    marker=".",
    color="k",
    label=f"mean(string{start_iteration}-{n_strings})",
)

for jj, j in enumerate(range(start_iteration, n_strings, n_average)):
    av_0 = np.mean(strings[j:, cv_0, :], axis=0)
    std_0 = np.std(strings[j:, cv_0, :], axis=0)
    av_1 = np.mean(strings[j:, cv_1, :], axis=0)
    std_1 = np.std(strings[j:, cv_1, :], axis=0)
    ax.errorbar(
        av_0, av_1, fmt="--", xerr=std_0, yerr=std_1, color=colors[jj], alpha=0.9
    )


ax.set_ylabel(
    f"{ndx_groups_list[cv_1*2]} - {ndx_groups_list[cv_1*2+1]} (nm)",
    size=18,
    labelpad=16,
)
ax.set_xlabel(
    f"{ndx_groups_list[cv_0*2]} - {ndx_groups_list[cv_0*2+1]} (nm)",
    size=18,
    labelpad=16,
)

ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.grid(which="minor")
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=11)
ax.legend()
cbar = colorbar(ax, cmap, norm, "iteration number", 20)

## Evolution over CVs that are a function of the cvs

If you are interested in studying the convergence of cvs that are a function of CVs (for example averaging over symmetrical distances). You can construct a `reduced_string` array in which cvs are a function of the cvs used for the string method. In the example bellow, we produce two cvs which are the mean of cvs used in the string method simulation. Then, similar plotting as before can be done. 

In addition if you are interested in the convergence of some other cv which is not a function of the cvs used in the string method you can also study them! Just extract the average value of that particular CV in the `md/*/*/restrained/traj_comp.xtc` for all the restrained simulation and shape them into an `reduced_string` numpy array with shape (n_iterations, n_cvs, n_beads).

If this sort of analysis is meaningless in your system, for example because the chosen cvs are very diagnostic, please ignore this section.

In [ ]:
reduced_string = np.hstack(
    [
        np.mean(strings[:, 0:2, :], axis=1, keepdims=True),
        np.mean(strings[:, 6:, :], axis=1, keepdims=True),
    ]
)
reduced_string_labels = ["X (nm)", "Y (nm)"]

In [ ]:
start_iteration = 100
n_average = 40

n_strings = strings.shape[0]
fig, ax = plt.subplots(1, 1, figsize=(10, 7))
cmap = plt.cm.viridis_r
n_colors = (n_strings - start_iteration) // n_average + 1
colors = cmap(np.linspace(0, 1, n_colors))  # yellow to blue
norm = mpl.colors.Normalize(vmin=start_iteration, vmax=n_strings)
ax.plot(
    reduced_string[0, 0, :],
    reduced_string[0, 1, :],
    ls=":",
    marker=".",
    label="string0",
    color="k",
)
av_0 = np.mean(reduced_string[start_iteration:, 0, :], axis=0)
std_0 = np.std(reduced_string[start_iteration:, 0, :], axis=0)
av_1 = np.mean(reduced_string[start_iteration:, 1, :], axis=0)
std_1 = np.std(reduced_string[start_iteration:, 1, :], axis=0)
ax.plot(
    av_0,
    av_1,
    ls="-",
    marker=".",
    color="k",
    label=f"mean(string{start_iteration}-{n_strings})",
)

for jj, j in enumerate(range(start_iteration, n_strings, n_average)):
    av_0 = np.mean(reduced_string[j:, 0, :], axis=0)
    std_0 = np.std(reduced_string[j:, 0, :], axis=0)
    av_1 = np.mean(reduced_string[j:, 1, :], axis=0)
    std_1 = np.std(reduced_string[j:, 1, :], axis=0)
    ax.errorbar(
        av_0, av_1, fmt="--", xerr=std_0, yerr=std_1, color=colors[jj], alpha=0.9
    )


ax.set_ylabel(
    reduced_string_labels[1], size=18, labelpad=16,
)
ax.set_xlabel(
    reduced_string_labels[0], size=18, labelpad=16,
)

ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.grid(which="minor")
ax.tick_params(axis="y", labelsize=14)
ax.tick_params(axis="x", labelsize=11)
ax.legend()
cbar = colorbar(ax, cmap, norm, "iteration number", 20)

## Visualize String

If you have been able to install correctly [NGLview](https://github.com/nglviewer/nglview) in this jupyter-lab, which is tricky sometimes, you can visualize the `md/iter/*/restrained/confout.gro` (in ribbon, green) files. `iteration` is the iteration you want to study

In [ ]:
iteration = 5
beads = natural_sort(glob.glob(f"md/{iteration}/*/restrained"))

In [ ]:
traj = mda.Universe(
    beads[0] + "/confout.gro", [b + "/confout.gro" for b in beads], in_memory=True
)
start = mda.Universe("topology/start.gro")
_ = AlignTraj(traj, start, select="name CA and protein", in_memory=True).run()

In [ ]:
view = nv.show_mdanalysis(traj, default=False)
view.add_ribbon(selection="protein", color="green", component=0)
view.center()

Sometimes need to run this cell several times to see something. Sometimes it takes a few seconds to load.

In [ ]:
view

# Free Energy Surface

Once your strings are converged, the swarms are sampling over and over the same part of phase space and we can discretrize it and do statistics on the jumps. This will result in a free energy surface along some cv, which may not need to be the ones that parametrize the string. It is very important to keep in mind that a converged string does not imply a converged FES and it might be necessary to do one or two (or more) hundred additional iterations.


Now instead of using the data in `strings/string*.xtx` we will use the data in `md/*/*/s*/pullx.xvg` if we want to use the cvs of the string. Otherwise, you add here code that reads `md/*/*/s*/traj_comp.xtc`, calculates your desired cv and then shapes the data into the correct shape `(n_iterations*n_swarms_per_iter*n_beads, n_frames_per_iter, n_cvs)`. 

The path of `sys.path.append` should lead to the library `string-method-gmxapi`  

In [ ]:
sys.path.append("../../")

from stringmethod.config import *
from stringmethod.postprocessing import *

In [ ]:
def load_swarm_data(extract, first_iteration=1, last_iteration=None):
    if last_iteration == None:
        last_iteration = sys.maxsize
    if extract:
        config = load_config("config.json")

        ce = CvValueExtractor.from_config(
            config=config,
            first_iteration=first_iteration,  # Exclude the first iterations to let the system equilibrate.
            last_iteration=last_iteration,  # Usefull to make blocks of the simulation
        )
        ce.run()
        ce.persist()
    return np.load("postprocessing/cv_coordinates.npy")

The `load_swarm_data` function will load the swarm data in the `cv_coordinates`. If you set `extract=True` it will read the data from the swarm files. If you have done this previously you can set `extract=False` so the function just reads `postprocessing/cv_coordinates.npy`. `first_iteration` can be used to exclude initial swarms as equilibration and `last_iteration` can be done to exclude some iterations for example if you want to estimate the FES convergence by comparing blocks of data.

In [ ]:
cv_coordinates = load_swarm_data(extract=True, first_iteration=1, last_iteration=None)

Since the FES that is easy to visualize can only be 1D or 2D we need to reduce the dimensionality of the CVs. For this there are several options:
+ You can choose a couple of cvs or one

In [ ]:
my_cvs = [0, 6]
cv_coordinates_clean = cv_coordinates[:, :, my_cvs]

+ You can do some function of the cvs, like the mean of several:

In [ ]:
cv_coordinates_clean = np.concatenate(
    [
        np.concatenate([cv_coordinates[:, :, 0:1], cv_coordinates[:, :, 1:2]], axis=0),
        np.concatenate([cv_coordinates[:, :, 6:7], cv_coordinates[:, :, 7:8]], axis=0),
    ],
    axis=2,
)

+ You can of course make your own function that extracts cvs from the trajectory and makes a cv_coordinates_clean with the correct shape.

Of the above cells only run the one you are more interested in.

In [ ]:
def calculate_transition_matrix(
    cv_coordinates, n_grid_points=15, T=300, kB=0.0019872041
):
    config = load_config("config.json")
    tc = TransitionCountCalculator.from_config(
        config=config,
        # You probably want to play around with n_grid_points.
        # It sets the resolution. Its optimal value depends on your swarm trajectory length and sample size
        n_grid_points=n_grid_points,
        cv_coordinates=cv_coordinates,
    )
    tc.run()
    tc.persist()
    fc = FreeEnergyCalculator.from_config(
        config=config, grid=tc.grid, transition_count=tc.transition_count
    )
    fc.run()
    fc.persist()
    return tc.grid, fc.free_energy

This functions takes the `cv_coordinates_clean` numpy array and calculates a transition matrix by doing a simple grid on the cv space. It also calculates the probability of transition using the master equation and this results in the calculation of a FES. `n_grid_points` choose the number of grid points of the grid, the coarse the grid the more detailed (and noisy) the FES. This parameter should be varied to obtain an acceptable signal-to-noise ration. The temperature `T` and the value of `kB` can be set too. `kB` is used to give units to the FES. In this example we will use "kBT" units since kBT=1.

In [ ]:
grid, free_energy = calculate_transition_matrix(
    cv_coordinates_clean, n_grid_points=20, T=1, kB=1
)

In [ ]:
def show_fes(grid, free_energy, fe_cut_off=None, cv_labels=["cv0 (nm)", "cv1 (nm)"], f_min=None, f_max=None):
    if fe_cut_off == None:
        fe_cut_off = sys.maxsize
    free_energy[free_energy > fe_cut_off] = np.nan
    cv_0 = grid[:, 0]
    fig, ax = plt.subplots(1, 1)
    if free_energy.shape[1] == 1:
        ax.plot(cv_0, free_energy, "--o")
        ax.set_ylabel("Free Energy (kT)")
    else:
        cv_1 = grid[:, 1]
        im = plt.contourf(
            cv_0,
            cv_1,
            free_energy.T,
            levels=20,
        #    norm=mpl.colors.PowerNorm(gamma=1 / 3),
            # Used to be rainbow cmap=plt.cm.rainbow
            cmap=plt.cm.viridis,
            vmin=f_min,
            vmax=f_max,
        )
        cbar = plt.colorbar(im)
        cbar.set_label("Free Energy (kT)")
        ax.set_ylabel(cv_labels[1])
        ax.yaxis.set_minor_locator(MultipleLocator(0.1))
        ax.yaxis.set_major_locator(MultipleLocator(0.1))
    ax.set_xlabel(cv_labels[0])
    ax.xaxis.set_minor_locator(MultipleLocator(0.1))
    ax.xaxis.set_major_locator(MultipleLocator(0.1))
    fig.tight_layout()
    return fig, ax

The function bellow takes the `grid` and `free_energy` arrays from the previous step and plots the free energy surface. The function returns the matplotlib `fig` and `ax` for you to format further if you want. `fe_cut_off` is a maximum value of free energy overwhich nothing is plotted and `cv_labels` are the labels of the cvs.

In [ ]:
fig, ax = show_fes(
    grid, free_energy, fe_cut_off=None, cv_labels=["cv0 (nm)", "cv1 (nm)"]
)

To save the plot.

In [ ]:
fig.savefig(f"{simulation_directory}/free_energy.svg", transparent=True)